In [1]:
from keras import Sequential
from keras.callbacks import History
from forecasting_functions import *

# importing the data
Stijn = True
fullYearpath = None
av_temperaturepath = None

if Stijn:
    fullYearpath = "D:\Onedrive\Leuven\Final project\data\Forecasting_writtendata\FullYear.csv"
    av_temperaturepath = "D:\Onedrive\Leuven\Final project\data\weather-avg.csv"
else:
    raise Exception("Put some paths here.")
fullYeardata = pd.read_csv(fullYearpath,index_col= "date",parse_dates= True)
av_temperature = pd.read_csv(av_temperaturepath,index_col='meter_id')
av_temperature = av_temperature.transpose()
av_temperature.index = pd.to_datetime(av_temperature.index)



name = fullYeardata.columns[1]
TS = fullYeardata[name]
temperature = av_temperature[name]



# all the training dates are present, dates from the test set can be missing.

# normalize the data --> min/max method (when using a single time serie)
temperature_norm,scaler_temperature = norm_forcast(temperature)
temperature_norm = substitute_missing_values_temperature(temperature_norm)
TS_norm,scaler_history = norm_forcast(TS)
temp = TS_norm[TS_norm.index.month == 1]
# temp = TS_norm[TS_norm.index.month != 12]
# test = TS_norm[TS_norm.index.month == 12]

# remove from the test set all the days that contain nan values -> only estimate real days
# test.dropna(inplace=True)

# substitute the missing values
temp = substitute_missing_values(temp)
training = temp[0:336]
validation = temp[336:384]
test = temp[384:528]
# training = temp[temp.index.month != 11]
# validation = temp[temp.index.month == 11]
TS_norm_full = substitute_missing_values(TS_norm.copy(deep=True))

amount of missing values: 0. 

amount of missing values: 0. 

amount of missing values: 0. 



In [112]:
X,y = input_output_LSTM(training, temperature_norm, 3)
X_val, y_val = input_output_LSTM(validation, temperature_norm, 3)
X.shape

(333, 3, 59)

In [4]:

from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(batch_shape=(1,3, 1))
lstm1, state_h, state_c = LSTM(1, stateful= True, return_state= True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
# make and show prediction
for row in data:
    row = row.reshape(1,3,1)
    print(model.predict(row))
    # model.reset_states()
    print(10*'-')

[array([[0.]], dtype=float32), array([[0.]], dtype=float32), array([[0.]], dtype=float32)]
----------
[array([[0.09549879]], dtype=float32), array([[0.09549879]], dtype=float32), array([[0.19883052]], dtype=float32)]
----------
[array([[0.01529508]], dtype=float32), array([[0.01529508]], dtype=float32), array([[0.03055247]], dtype=float32)]
----------
[array([[0.00229115]], dtype=float32), array([[0.00229115]], dtype=float32), array([[0.00458128]], dtype=float32)]
----------
[array([[0.19473065]], dtype=float32), array([[0.19473065]], dtype=float32), array([[0.43662012]], dtype=float32)]
----------


In [5]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_state= True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
data = data.reshape((5,3,1))
# make and show prediction
# for row in data:
#     row = row.reshape(1,3,1)
output = model.predict(data)
print(model.predict(data))
print(10*'-')


[array([[0.        ],
       [0.14619657],
       [0.        ],
       [0.        ],
       [0.3154364 ]], dtype=float32), array([[0.        ],
       [0.14619657],
       [0.        ],
       [0.        ],
       [0.3154364 ]], dtype=float32), array([[0.        ],
       [0.26257628],
       [0.        ],
       [0.        ],
       [0.53567994]], dtype=float32)]
----------


In [6]:

from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array

# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))


[array([[0.05716937]], dtype=float32), array([[0.05716937]], dtype=float32), array([[0.11988591]], dtype=float32)]


In [7]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 1))
lstm1 = LSTM(1, stateful= False)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)
# define input data
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
# make and show prediction
# for row in data:
#     row = row.reshape(5,3,1)
print(model.predict(data.reshape((5,3,1)), batch_size= 5))
# model.reset_states()
print(10*'-')

TypeError: Input() got an unexpected keyword argument 'input_shape'

In [16]:
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
model = Sequential()
model.add(LSTM(1, batch_input_shape=(5,3,1)))

# for row in data:
#     row = row.reshape(1,3,1)
#     print(model.predict(row))
#     # model.reset_states()
#     print(10*'-')

data = data.reshape((5,3,1))
print(model.predict(data, batch_size= 5))


[[0.        ]
 [0.0141051 ]
 [0.        ]
 [0.        ]
 [0.03010445]]


In [158]:
from keras.layers import Dense,  LSTM,Embedding
from keras.models import Sequential, save_model, load_model
from keras import regularizers
from keras.callbacks import EarlyStopping,ModelCheckpoint, History

def build_model_stateless1(training: pd.Series,validation: pd.Series, temperature_norm: pd.Series, lag_value: int,nb_epoch: int = 1, regularization_parameter: float = 0.01,batch_size_para: int = 32,verbose_para: int = 1, save: bool = False):
    history = History()
    X,y = input_output_LSTM(training, temperature_norm, lag_value)
    X_val,y_val = input_output_LSTM(validation, temperature_norm, lag_value)
    print(len(X), len(X_val))
    regularizers.l2(l=regularization_parameter)
    model = Sequential()
    # no initial state is given --> hidden state are tensors filled with zeros
    # dropout=None,recurrent_dropout=None
    model.add(LSTM(units=1,activation='tanh',stateful= False, batch_input_shape=(8,X.shape[1],X.shape[2])))
    # model.add(Dense(units=20,activation='relu',kernel_regularizer='l2'))
    # model.add(Dropout(0.10))
    # model.add(Dense(units=y.shape[1],activation='relu',kernel_regularizer='l2'))
    model.compile(optimizer='adam',loss='mse')
    early_stopping_monitor = EarlyStopping(patience=5,restore_best_weights=True)
    # shuffle false --> not shuffle the training data
    model.fit(x=X,y=y,validation_data=(X_val,y_val), epochs=nb_epoch,shuffle= False, batch_size=8,callbacks=[early_stopping_monitor,history],verbose=verbose_para)
    # save the trained_model

    return model

trained_model = build_model_stateless1(training, validation,temperature_norm,3,3,verbose_para=1)

333 45
Train on 333 samples, validate on 45 samples
Epoch 1/3
232/333 [===================>..........] - ETA: 0s - loss: 0.1265 

InvalidArgumentError: Incompatible shapes: [8,1] vs. [5,1]
	 [[{{node training_29/Adam/gradients/loss_33/lstm_47_loss/mean_squared_error/sub_grad/BroadcastGradientArgs}}]]

In [153]:
X_pred = X
X_pred = np.insert(X_pred,0,0,axis=0)
X_pred = np.insert(X_pred,2,0,axis=0)
X_pred = np.insert(X_pred,-1,0,axis=0)
# trained_model.reset_states()
# for i in range(len(X_pred)):
#     row = X_pred[i,:,:]
#     row = np.expand_dims(row,axis=0)
#     print(trained_model.predict(row, batch_size= 2))
print(X_pred[0:3].shape)
print(trained_model.predict(X_pred[0:3], batch_size= 3))

(3, 3, 59)
[[0.07492943]
 [0.1329707 ]
 [0.05895755]]
